In [197]:
using ClimaCore
using ClimaCore: Spaces, Geometry, Fields
ENV["TEST_NAME"]="sphere/nonhydrostatic_gravity_wave";

In [10]:
include("driver.jl")

252.094036 seconds (90.90 M allocations: 5.990 GiB, 0.65% gc time, 19.35% compilation time)
elapsed time (ns): 252094035998
gc time (ns):      1639035971
bytes allocated:   6432117837
pool allocs:       90848223
non-pool GC allocs:45651
malloc() calls:    1253
realloc() calls:   17
GC pauses:         16
full collections:  1


[ Info: Precompiling ClimaCorePlots [cf7c7e5a-b407-4c48-9047-11a94a308626]
[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
[ Info: Precompiling ClimaCoreVTK [c8b6d40d-e815-466f-95ae-c48aefa668fa]
[ Info: Running sphere/nonhydrostatic_gravity_wave
[ Info: L₂ norm of ρe at t = 0.0: 28851.938409838014
[ Info: L₂ norm of ρe at t = 432000.0: 28286.204772917376
┌ Info: Saved animation to
└   fn = "/Users/asridhar/Research/Codes/ClimaCore.jl/examples/hybrid/sphere/output/nonhydrostatic_gravity_wave/v.mp4"
┌ Info: Saved animation to
└   fn = "/Users/asridhar/Research/Codes/ClimaCore.jl/examples/hybrid/sphere/output/nonhydrostatic_gravity_wave/u.mp4"
┌ Info: Saved animation to
└   fn = "/Users/asridhar/Research/Codes/ClimaCore.jl/examples/hybrid/sphere/output/nonhydrostatic_gravity_wave/w.mp4"
┌ Info: Saved animation to
└   fn = "/Users/asridhar/Research/Codes/ClimaCore.jl/examples/hybrid/sphere/output/nonhydrostatic_gravity_wave/T.mp4"


In [198]:
using ClimaCoreMakie
using GLMakie, WGLMakie

[ Info: Precompiling WGLMakie [276b4fcb-3e11-5398-bf8b-a0c2d153d008]


In [269]:
function zonal_velocity(time_index, lev=5, sol=sol)
    zv = @. Geometry.UVVector(sol.u[time_index].c.uₕ).components.data.:1
    return ClimaCore.Spaces.level(zv,lev)
end
function meridional_velocity(time_index, lev=5, sol=sol)
    mv = @. Geometry.UVVector(sol.u[time_index].c.uₕ).components.data.:2
    return ClimaCore.Spaces.level(zv,lev)
end
function vertical_velocity(time_index, lev=5, sol=sol)
    ᶠw = @. Geometry.WVector(sol.u[time_index].f.w).components.data.:1
    ᶜw = @. ᶜinterp(ᶠw)
    return ClimaCore.Spaces.level(ᶜw,lev)
end
function temperature_perturbation(time_index, lev=5, sol=sol)
     Y = sol.u[time_index]
    ᶜρ = Y.c.ρ
    ᶜu = Geometry.UVVector.(Y.c.uₕ).components.data.:1
    ᶜv = Geometry.UVVector.(Y.c.uₕ).components.data.:2
    ᶠw = Geometry.WVector.(Y.f.w).components.data.:1
    ᶜw = ᶜinterp.(ᶠw)
    ᶜuvw = @. Geometry.UVWVector(Y.c.uₕ) + Geometry.UVWVector(ᶜinterp(Y.f.w))
    ᶜz = ᶜlocal_geometry.coordinates.z
    eint = @. Y.c.ρe / ᶜρ - (grav * ᶜz) - 1/2 * norm_sqr(ᶜuvw)
    T = @. eint / cv_d + T_tri 
    return ClimaCore.Fields.level(T .- 300,lev)
end
ind_endtime = length(sol)
@info "Number of timesteps available is $(ind_endtime))"

[ Info: Number of timesteps available is 481)


In [270]:
maximum(temperature_perturbation(400))

-35.70223951640344

In [271]:
# animation settings
framerate = 10
time_iterator = range(1, ind_endtime)
fig = ClimaCoreMakie.viz(zonal_velocity(1)).figure
record(fig, "zonal_velocity.mp4", time_iterator; framerate = framerate) do timeind 
    ClimaCoreMakie.viz(zonal_velocity(timeind))
end

JSServe.App(WGLMakie.var"#145#146"{Scene, Base.RefValue{Any}}(Scene (800px, 600px):
  0 Plots
  1 Child Scene:
    └ Scene (768px, 568px), Base.RefValue{Any}(nothing)))

"zonal_velocity.mp4"

In [290]:
zvfig = Figure()
zvax = Axis(zvfig[1, 1])
sl_t = Slider(zvfig[2,1], range = 1:1:ind_endtime, startvalue = 1)
sl_z = Slider(zvfig[1,2], range = 1:1:10, horizontal = false, startvalue = 1)

Slider()

In [291]:
dyn_temp = Makie.lift(sl_t.value, sl_z.value) do it, iz
    temperature_perturbation(it, iz)
end
ClimaCoreMakie.viz!(dyn_temp)

Combined{ClimaCoreMakie.viz, Tuple{ClimaCore.Fields.Field{IJFH{Float64, 5, SubArray{Float64, 4, Array{Float64, 5}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}}, true}}, ClimaCore.Spaces.SpectralElementSpace2D{ClimaCore.Topologies.Topology2D{ClimaCore.Meshes.EquiangularCubedSphere{ClimaCore.Domains.SphereDomain{Float64}, ClimaCore.Meshes.NormalizedBilinearMap}, CartesianIndices{3, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}}, LinearIndices{3, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}}, NamedTuple{(), Tuple{}}}, ClimaCore.Spaces.Quadratures.GLL{5}, ClimaCore.Geometry.SphericalGlobalGeometry{Float64}, IJFH{ClimaCore.Geometry.LocalGeometry{(1, 2, 3), ClimaCore.Geometry.LatLongZPoint{Float64}, Float64, StaticArrays.SMatrix{3, 3, Float64, 9}}, 5, SubArray{Float64, 4, Array{Float64, 5}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Ba